In [4]:
import tensorflow as tf
import numpy as np
from models.GoogleNet import GoogleNet_Inception_v2, GoogleNet_Inception_v3, GoogleNet_Inception_v4
from models.InceptionResNet import Inception_ResNet_v2
import os

num_epochs = 8
batch_size = 32
learning_rate = 0.001
data_dir = './data/cats-vs-dogs/'
train_dir = data_dir + 'train/'
test_dir = data_dir + 'valid/'
checkpoint_path = "training_cats_vs_dogs_inception_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

class FixedImageDataGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def standardize(self, x):
        if self.featurewise_center:
            x = ((x/255.) - 0.5) * 2.
        return x

def train():
    datagen = FixedImageDataGenerator(
        rescale=1./255,
        #featurewise_center=True,  # 数据集去中心化
        #rotation_range=10,        # 旋转角度范围
        #width_shift_range=0.1,    # 水平位置平移
        #height_shift_range=0.1   # 上下位置平移
        #horizontal_flip=True,   # 随机水平翻转
        #fill_mode='nearest'     # 填充模式
        )
    test_datagen =  FixedImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(299, 299),  # all images will be resized to 299x299
        batch_size=batch_size,
        class_mode='sparse')  # since we use sparse_categorical_crossentropy loss, we need categorical labels
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='sparse')
    
    history = model.fit_generator(
         train_generator,
         steps_per_epoch=23000 / batch_size,
         epochs=num_epochs,
         verbose=1,
         validation_data=test_generator,
         callbacks=[cp_callback_mc, reduce_lr, earlystop_callback]
         )
    return history

    
if __name__ ==  '__main__':
    model = GoogleNet_Inception_v2(2)
    model.build(input_shape=(None, 299, 299, 3))
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    cp_callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                        save_weights_only=True,
                                                        verbose=0)
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    if(latest != None):
        model.load_weights(latest) 
    #log_dir = "./tensorboard"
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)   #20epoch val_loss没有改善停止运行
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     patience=5,
                                                     factor=0.2,  #降低因子，每次降低*0.2
                                                     cooldown=5, #冷却时间5epoch
                                                     verbose=1)   # 设置学习率衰减，经过5epoch，loss未衰减学习率*0.1

Model: "inception__res_net_v2_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
google_net__basic_block_stem multiple                  608320    
_________________________________________________________________
sequential_9 (Sequential)    multiple                  699120    
_________________________________________________________________
inception__res_net__reductio multiple                  2906368   
_________________________________________________________________
sequential_10 (Sequential)   multiple                  11785280  
_________________________________________________________________
inception__res_net__reductio multiple                  3936256   
_________________________________________________________________
sequential_11 (Sequential)   multiple                  12506560  
_________________________________________________________________
global_average_pooling2d_3 ( multiple      

In [ ]:
    history = train()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['sparse_categorical_accuracy']
val_acc=history.history['val_sparse_categorical_accuracy']

epochs = range(len(train_acc))

plt.plot(epochs,train_loss,'r', label='train_loss')
plt.plot(epochs,val_loss,'b', label='val_loss')
plt.title('train_loss vs val_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.figure()

plt.plot(epochs,train_acc,'r', label='train_acc')
plt.plot(epochs,val_acc,'b', label='val_acc')
plt.title('train_acc vs val_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.figure()